In [1]:
using Pkg
Pkg.activate(".")

"/home/michel/git/structjump-demo/Project.toml"

In [2]:
using StructJuMP
using StructJuMPSolverInterface
include("acopf.jl")

acopf_initialPt_IPOPT (generic function with 1 method)

In [3]:
opfdata = opf_loaddata("data/case9")


OPFData(Bus[Bus(1, 3, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(2, 2, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(3, 2, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(4, 1, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(5, 1, 90.0, 30.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(6, 1, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(7, 1, 100.0, 35.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(8, 1, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(9, 1, 125.0, 50.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9)], Line[Line(1, 4, 0.0, 0.0576, 0.0, 250.0, 250.0, 250.0, 0.0, 0.0, 1, -360.0, 360.0), Line(4, 5, 0.017, 0.092, 0.158, 250.0, 250.0, 250.0, 0.0, 0.0, 1, -360.0, 360.0), Line(5, 6, 0.039, 0.17, 0.358, 150.0, 150.0, 150.0, 0.0, 0.0, 1, -360.0, 360.0), Line(3, 6, 0.0, 0.0586, 0.0, 300.0, 300.0, 300.0, 0.0, 0.0, 1, -360.0, 360.0), Line(6, 7, 0.0119, 0.1008, 0.209, 150.0, 150.0, 150.0, 0.0, 0.0, 1, -360.0, 3

This prints the loaded data `opfdata::OPFData`
```julia
mutable struct OPFData
  buses::Array{Bus}
  lines::Array{Line}
  generators::Array{Gener}
  bus_ref::Int
  baseMVA::Float64
  BusIdx::Dict{Int,Int}    #map from bus ID to bus index
  FromLines::Array         #From lines for each bus (Array of Array)
  ToLines::Array           #To lines for each bus (Array of Array)
  BusGenerators::Array     #list of generators for each bus (Array of Array)
end

```

In [4]:
opfmodel = acopf_model(opfdata)

Buses: 9  Lines: 9  Generators: 3
Lines with limits  9


Minimization problem with:
 * 0 linear constraints
 * 36 nonlinear constraints
 * 24 variables
Solver is default solver

The output is a fill OPF model of the IEEE 9-bus system.

We still have to initialize the model variables. We access the first-stage variables active and reactive power constraints plus the voltages and set them to initial values that are computed in `acopf_initialPt_IPOPT`.

In [5]:
Pg0,Qg0,Vm0,Va0 = acopf_initialPt_IPOPT(opfdata)
setvalue(opfmodel[:Pg], Pg0)  
setvalue(opfmodel[:Qg], Qg0)
setvalue(opfmodel[:Vm], Vm0)
setvalue(opfmodel[:Va], Va0)

Finally we solve the model using Ipopt.

In [6]:
solve(opfmodel;solver="Ipopt")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      110
Number of nonzeros in inequality constraint Jacobian.:       70
Number of nonzeros in Lagrangian Hessian.............:      465

Total number of variables............................:       23
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       15
                     variables with only upper bounds:        0
Total number of equ

:Solve_Succeeded

In [7]:
acopf_outputAll(opfmodel,opfdata)

Objective value: 0USD/hr
============================= BUSES ==================================
  BUS    Vm     Va   |   Pg (MW)    Qg(MVAr) 
                     |     (generation)      
----------------------------------------------------------------------
   1 |   1.10    0.00 |   89.80       12.97  | 
   2 |   1.10    4.89 |  134.32        0.03  | 
   3 |   1.09    3.25 |   94.19      -22.63  | 
   4 |   1.09   -2.46 |    --          --    | 
   5 |   1.08   -3.98 |    --          --    | 
   6 |   1.10    0.60 |    --          --    | 
   7 |   1.09   -1.20 |    --          --    | 
   8 |   1.10    0.91 |    --          --    | 
   9 |   1.07   -4.62 |    --          --    | 


Number of lines with flow limits: 9
================ Lines within 20 % of flow capacity ===================
Line   From Bus    To Bus    At capacity
  6        7        8        83.546%
  9        9        4        81.473%



signal (15): Terminated
in expression starting at /home/michel/.julia/packages/IJulia/gI2uA/src/kernel.jl:52
